In [ ]:
import sys
sys.path.insert(0, '..')

import os.path as op

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

import numpy as np
import pandas as pd
import pickle

from paus_utils import *
from jpasLAEs.utils import bin_centers

from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord

In [ ]:
### LOAD STUFF ###
region_list = ['W1', 'W2', 'W3']

# SDSS Xmatch
sdss_xm_list = []
for jj, field_name in enumerate(region_list):
    pathname = f'/home/alberto/almacen/PAUS_data/catalogs/Xmatch_SDSS_{field_name}.csv'
    this_sdss_xm = pd.read_csv(pathname)
    this_sdss_xm = this_sdss_xm.drop('Unnamed: 0', axis=1)
    this_sdss_xm = this_sdss_xm.reset_index(drop=True)
    this_sdss_xm['ref_id'] = this_sdss_xm['ref_id']
    sdss_xm_list.append(this_sdss_xm)
sdss_xm = pd.concat(sdss_xm_list)
sdss_xm = sdss_xm.drop_duplicates(subset='ref_id')
sdss_xm = sdss_xm.reset_index(drop=True)

# Load HETDEX
path_to_cat = '/home/alberto/almacen/HETDEX_catalogs/hetdex_source_catalog_1'
version = 'v3.2'

hetdex_table = pd.DataFrame(
    dict(Table.read(op.join(path_to_cat, f'hetdex_sc1_{version}.ecsv')))
    )

# PAUS selection
sel_dfs = []
LFs_dir = '/home/alberto/almacen/PAUS_data/Lya_LFs'
for i in range(19):
    for jj, region in enumerate(region_list):
        with open(f'{LFs_dir}/Lya_LF_nb{i}-{i}_{region}/selection.pkl', 'rb') as file:
            this_dict = pickle.load(file)
            # Separate the L_lya err into two columns
            this_dict['L_lya_corr_err_up'] = this_dict['L_lya_corr_err'][1]
            this_dict['L_lya_corr_err_down'] = this_dict['L_lya_corr_err'][0]
            this_dict['ref_id'] = this_dict['ref_id']
            del this_dict['L_lya_corr_err']
        
        sel_dfs.append(pd.DataFrame(this_dict))
selection = pd.concat(sel_dfs)
selection['ref_id'] = selection['ref_id'].astype(int)
selection = selection.drop_duplicates(subset='ref_id')
selection = selection.reset_index(drop=True)

# Join selection with sdss_xm
selection = pd.merge(selection, sdss_xm, on='ref_id', how='left',
                     suffixes=['', '_SDSS'])

selection['z_NB'] = z_NB(selection['lya_NB'])

# Define nice_z
nice_z = np.array(
    (np.abs(selection['z_best'] - z_NB(selection['lya_NB'])) < 0.2)
)
selection['nice_z'] = nice_z


# Load Chandra CSC 2.0 Xmatch
csc = pd.read_csv('/home/alberto/almacen/PAUS_data/Xmatch/LAE_selection_Chandra_CSC_2.0.csv')
csc = csc.replace('         ', np.nan)
csc['flux_aper_b'] = csc['flux_aper_b'].astype(float)
csc['flux_aper_lolim_b'] = csc['flux_aper_lolim_b'].astype(float)
csc['flux_aper_hilim_b'] = csc['flux_aper_hilim_b'].astype(float)
selection = pd.merge(selection,
                     csc[['usrid', 'flux_aper_b', 'flux_aper_lolim_b', 'flux_aper_hilim_b']],
                     left_on='ref_id', right_on='usrid', how='left')
selection = selection.drop('usrid', axis=1)

# Xmatch
coords_paus = SkyCoord(ra=np.array(selection['RA']) * u.deg,
                       dec=np.array(selection['DEC']) * u.deg)
coords_hetdex = SkyCoord(ra=np.array(hetdex_table['RA']) * u.deg,
                         dec=np.array(hetdex_table['DEC']) * u.deg)
                    
xm_id, ang_dist, _= coords_paus.match_to_catalog_sky(coords_hetdex)

# Objects with 1 arcsec of separation
mask_dist = (ang_dist <= 1.5 * u.arcsec)

selection['z_HETDEX'] = np.array(hetdex_table['z_hetdex'])[xm_id]
selection['z_HETDEX'][~mask_dist] = -1
selection['z_HETDEX_conf'] = np.array(hetdex_table['z_hetdex_conf'])[xm_id]
selection['z_HETDEX_conf'][~mask_dist] = -1
selection['HETDEX_class'] = np.array(hetdex_table['source_type'])[xm_id]
selection['HETDEX_class'][~mask_dist] = ''


# Save the selection
save_to_path = '/home/alberto/almacen/PAUS_data/catalogs/'
selection.to_csv(f'{save_to_path}/LAE_selection.tsv', sep='\t')

selection

In [ ]:
# Plot EWs
fig, ax = plt.subplots(figsize=(6, 4))

log_EW_bins = np.linspace(1, 4, 20 + 1)
log_EW_bin_c = bin_centers(log_EW_bins)
ew_h, _ = np.histogram(np.log10(selection['EW0_lya']), log_EW_bins)
ew_h = ew_h / ew_h.sum()

ax.plot(log_EW_bin_c, ew_h, drawstyle='steps-mid',
        lw=2)

ax.set_ylabel('Fraction of sources', fontsize=16)
ax.set_xlabel(r'log$_{10}$ EW$_0^{\mathrm{Ly}\alpha}$ [\AA]', fontsize=16)

ax.tick_params(labelsize=14, direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

ax.set_xlim(1, 4)

plt.show(block=False)

In [ ]:
# EW

fig, ax = plt.subplots(figsize=(6, 4))

mask = nice_z & (selection['EW0_lya_SDSS'] > 0)
ax.errorbar(selection['EW0_lya_SDSS'][mask], selection['EW0_lya'][mask],
           xerr=np.abs(selection['EW0_lya_err_SDSS'][mask]),
           yerr=selection['EW0_lya_err'][mask],
           fmt='o', ms=4, c='k', ecolor='k')

ax.plot([0.01, 1e6], [0.01, 1e6],
        ls='--', c='r', zorder=-99, label='1:1')

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlim(3, 10000)
ax.set_ylim(10, 300)

ax.tick_params(labelsize=16, direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

ax.set_ylabel('Narrow Band EW$_0$ [\AA]')
ax.set_xlabel('Spectroscopic EW$_0$ [\AA]')

plt.show(block=False)

In [ ]:
# L_Lya

fig, ax = plt.subplots(figsize=(6, 4))

mask = nice_z & (selection['L_lya_SDSS'] > 40)
ax.errorbar(selection['L_lya_SDSS'][mask], selection['L_lya_corr'][mask],
            yerr=[selection['L_lya_corr_err_up'][mask],
                  selection['L_lya_corr_err_down'][mask]],
            xerr=[selection['L_lya_err_up'][mask],
                  selection['L_lya_err_down'][mask]],
            fmt='o', ms=3, c='k')

ax.plot([0.01, 1e6], [0.01, 1e6],
        ls='--', c='r', zorder=-99, label='1:1')

ax.set_xlim(42.9, 45.6)
ax.set_ylim(43.5, 45.5)

ax.tick_params(labelsize=16, direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

ax.set_ylabel(r'Narrow Band $\log_{10}(L_{\mathrm{Ly}\alpha}/\mathrm{erg\,s}^{-1})$')
ax.set_xlabel(r'Spectroscopic $\log_{10}(L_{\mathrm{Ly}\alpha}/\mathrm{erg\,s}^{-1})$')

plt.show(block=False)


###############################################3

fig, ax = plt.subplots(figsize=(6, 4))

L_diff = 10**selection['L_lya_corr'] - 10**selection['L_lya_SDSS']
L_diff_err_nb = 10**(selection['L_lya_corr'] + selection['L_lya_corr_err_up']) - 10**selection['L_lya_corr']
L_diff_err_sp = 10**(selection['L_lya_SDSS'] + selection['L_lya_err_up']) - 10**selection['L_lya_SDSS']
L_diff_err = (L_diff_err_nb ** 2 + L_diff_err_sp ** 2) ** 0.5

ax.hist((L_diff / L_diff_err)[mask], np.linspace(-10, 10, 40))
print(f'(mean, std) = {np.nanmedian((L_diff / L_diff_err)[mask]):0.3f}, '
      f'{np.nanstd((L_diff / L_diff_err)[mask]):0.3f}')

plt.show(block=False)

In [ ]:
from jpasLAEs.utils import smooth_hist

mask_L = (selection['L_lya_corr'] > 43) & (selection['EW0_lya'] > 00)
sel_mask = (selection['z_best'] > 0) & mask_L
nice_mask = sel_mask & nice_z

fig, ax = plt.subplots(figsize=(6, 4))

nice_h_smooth, to_plot_c = smooth_hist(selection['z_NB'][nice_mask],
                                       2.7, 4.5, 0.1, 0.3)
sel_h_smooth, to_plot_c = smooth_hist(selection['z_NB'][sel_mask],
                                      2.7, 4.5, 0.1, 0.3)

sdss_p = nice_h_smooth / sel_h_smooth
p_err = ((nice_h_smooth ** 0.5 / sel_h_smooth) ** 2
         + (sdss_p * sel_h_smooth ** -0.5) ** 2) ** 0.5
ax.errorbar(to_plot_c, sdss_p, lw=2, label='L_lya$>$43, EW0$>$0')
            # yerr=p_err)

mask_L = (selection['L_lya_corr'] > 44) & (selection['EW0_lya'] > 00)
sel_mask = (selection['z_best'] > 0) & mask_L
nice_mask = sel_mask & nice_z

nice_h_smooth, to_plot_c = smooth_hist(selection['z_NB'][nice_mask],
                                       2.7, 4.5, 0.1, 0.3)
sel_h_smooth, to_plot_c = smooth_hist(selection['z_NB'][sel_mask],
                                      2.7, 4.5, 0.1, 0.3)

sdss_p = nice_h_smooth / sel_h_smooth
p_err = ((nice_h_smooth ** 0.5 / sel_h_smooth) ** 2
         + (sdss_p * sel_h_smooth ** -0.5) ** 2) ** 0.5
ax.errorbar(to_plot_c, sdss_p, lw=2, label='L_lya$>$44, EW0$>$0')

mask_L = (selection['L_lya_corr'] > 44) & (selection['EW0_lya'] > 50)
sel_mask = (selection['z_best'] > 0) & mask_L
nice_mask = sel_mask & nice_z

nice_h_smooth, to_plot_c = smooth_hist(selection['z_NB'][nice_mask],
                                       2.7, 4.5, 0.1, 0.3)
sel_h_smooth, to_plot_c = smooth_hist(selection['z_NB'][sel_mask],
                                      2.7, 4.5, 0.1, 0.3)

sdss_p = nice_h_smooth / sel_h_smooth
p_err = ((nice_h_smooth ** 0.5 / sel_h_smooth) ** 2
         + (sdss_p * sel_h_smooth ** -0.5) ** 2) ** 0.5
ax.errorbar(to_plot_c, sdss_p, lw=2, label='L_lya$>$44, EW0$>$50')
            # yerr=p_err)

ax.legend()

ax.set_ylim(0, 1.01)
ax.set_xlabel('Redshift')
ax.set_ylabel('SDSS Purity')

plt.show(block=False)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3.5))

conf_mask = selection['z_HETDEX_conf'] >= 0.9

mask = (selection['z_best'] > 0) & (selection['L_lya_corr'] > 0)
ax.plot(selection['z_best'][mask], selection['z_NB'][mask],
        ls='', marker='o', ms=6, mew=1.5,
        mec='g', mfc='none')
mask = (selection['z_HETDEX'] > 0) & (selection['L_lya_corr'] > 40) & conf_mask
ax.plot(selection['z_HETDEX'][mask], selection['z_NB'][mask],
        ls='', marker='^', ms=6, mew=1.5,
        mec='peru', mfc='none')

# lines
zsp_xx = np.linspace(0, 5, 50)
w_lya = 1215.67
w_CIV = 1549.48
w_CIII = 1908.73
w_MgII = 2799.12
w_OII = 3727
w_OIII = (4960 + 5008) * 0.5
w_Hbeta = 4862
ax.plot(zsp_xx, zsp_xx, ls='-', c='r', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_CIV - 1, zsp_xx, 
        ls='--', c='dimgray', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_CIII - 1, zsp_xx, 
        ls='--', c='dimgray', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_MgII - 1, zsp_xx, 
        ls='--', c='dimgray', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_OII - 1, zsp_xx, 
        ls='--', c='dimgray', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_OIII - 1, zsp_xx, 
        ls='--', c='dimgray', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_Hbeta - 1, zsp_xx, 
        ls='--', c='dimgray', zorder=-99)

text_plot = [[0.3, 3.9, '[OII]'],
             [0.01, 4.4, '[OIII]'],
             [0.37, 4.3, r'H$\beta$'],
             [0.8, 3.9, 'MgII'],
             [1.8, 3.9, 'CIII]'],
             [2.6, 3.9, 'CIV'],
             [3.5, 3.9, r'\bf Ly$\alpha$']]
for text in text_plot:
        ax.text(text[0], text[1], text[2], fontsize=11,
                zorder=99999)

ax.set_xlim(0, 4.5)
ax.set_ylim(2.7, 4.5)

ax.set_ylabel('$z_\mathrm{NB}$', fontsize=15)
ax.set_xlabel('$z_\mathrm{spec}$', fontsize=15)
ax.tick_params(direction='in', which='both', labelsize=13)
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

plt.show(block=False)

In [ ]:
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

best_z = np.array(selection['z_NB'])
best_z[selection['z_best'] > 0] = selection['z_best'][selection['z_best'] > 0]

Dl = cosmo.luminosity_distance(best_z).to(u.cm).value

# L_xray = np.log10(selection['flux_aper_b'] * 4 * np.pi * Dl**2)
# L_xray_upper = np.log10(selection['flux_aper_hilim_b'] * 4 * np.pi * Dl**2)
# L_xray_lower = np.log10(selection['flux_aper_lolim_b'] * 4 * np.pi * Dl**2)

L_xray = selection['flux_aper_b']
L_xray_upper = selection['flux_aper_hilim_b']
L_xray_lower = selection['flux_aper_lolim_b']



fig, ax = plt.subplots(figsize=(6, 4))

mask = selection['z_best'] > 2.7
ax.errorbar(selection['L_lya_corr'][mask], L_xray[mask],
            xerr=[selection['L_lya_corr_err_up'][mask],
                  selection['L_lya_corr_err_down'][mask]],
            yerr=[(L_xray_upper - L_xray)[mask],
                  (L_xray - L_xray_lower)[mask]],
            fmt='o', c='k', label='LAEs')
mask = selection['z_best'] < 0
ax.errorbar(selection['L_lya_corr'][mask], L_xray[mask],
            xerr=[selection['L_lya_corr_err_up'][mask],
                  selection['L_lya_corr_err_down'][mask]],
            yerr=[(L_xray_upper - L_xray)[mask],
                  (L_xray - L_xray_lower)[mask]],
            fmt='o', c='gray', label='No spec')
mask = (selection['z_best'] > 0) & (selection['z_best'] < 2.7)
ax.errorbar(selection['L_lya_corr'][mask], L_xray[mask],
            xerr=[selection['L_lya_corr_err_up'][mask],
                  selection['L_lya_corr_err_down'][mask]],
            yerr=[(L_xray_upper - L_xray)[mask],
                  (L_xray - L_xray_lower)[mask]],
            fmt='o', c='r', zorder=-99, label='Cont.')

ax.legend()

ax.set_xlabel(r'$\log_{10}(L_{\mathrm{Ly}\alpha}/\mathrm{erg\,s}^{-1})$')
ax.set_ylabel('Chandra b flux [erg\,s$^{-1}$\,cm$^{-2}$]')
ax.set_yscale('log')

plt.show(block=False)

In [ ]:
mask = np.isfinite(selection['L_lya_corr']) & np.isfinite(L_xray)
np.corrcoef(selection['L_lya_corr'][mask], L_xray[mask])

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

nb_list = [[0, 2], [2, 4], [4, 6], [6, 8],
           [8, 10], [10, 12], [12, 14], [14, 16],
           [16, 18]]

for [nb1, nb2] in nb_list:
    mask = (selection['lya_NB'] >= nb1) & (selection['lya_NB'] <= nb2)
    ax.scatter(selection['RA'][mask], selection['DEC'][mask],
               s=20, label=f'nb{nb1}-{nb2}')

ax.set_ylim(51.5, 55.5)
ax.set_xlim(209, 220)

ax.legend()

plt.show()